# 📥 Importing Libraries

Data Mining Assignment 2 

Aarona Preethee P

1002029616 

In [1]:
import re
import json
import string
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# 🗃️ Load Dataset

In [2]:
path ='../input/news-category-dataset/News_Category_Dataset_v2.json'

In [3]:
list_ = []
with open(path) as files:
    for file in files:
        list_.append(json.loads(file))

# 📝 Meta information of Dataframe

In [4]:
data = pd.DataFrame(list_)
data.head()

In [5]:
data.info()

# 🔎 Checking for NaN values

In [6]:
data.isnull().sum()

# 🔥 EDA & Visualization

In [7]:
plt.figure(figsize=(14,8))
count = data.category.value_counts()
sns.barplot(x=count.index, y=count)
plt.xlabel('Category')
plt.ylabel('Count')
plt.xticks(rotation=90);

# 🧹 Cleaning Data

In [9]:
def clean_text(text):
    text = text.lower()                                  # lower-case all characters
    text =  re.sub(r'@\S+', '',text)                     # remove twitter handles
    text =  re.sub(r'http\S+', '',text)                  # remove urls
    text =  re.sub(r'pic.\S+', '',text) 
    text =  re.sub(r"[^a-zA-Z+']", ' ',text)             # only keeps characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text+' ')      # keep words with length>1 only
    text = "".join([i for i in text if i not in string.punctuation])
    words = nltk.tokenize.word_tokenize(text)
    stopwords = nltk.corpus.stopwords.words('english')   # remove stopwords
    text = " ".join([i for i in words if i not in stopwords and len(i)>2])
    text= re.sub("\s[\s]+", " ",text).strip()            # remove repeated/leading/trailing spaces
    return text

In [10]:
data['Text_cleaning'] = data.headline.apply(clean_text)
data.head()

In [14]:
vectorizer = CountVectorizer()
data_vectorizer = vectorizer.fit_transform(data['Text_cleaning'])

In [16]:
labels = data['category']

# ✂️ Train test split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(data_vectorizer, labels, test_size=0.2, random_state=42)

# 📚 Training model

In [19]:
nb = MultinomialNB()
nb.fit(X_train,y_train)

In [20]:
y_pred = nb.predict(X_test)

# 🧪 Test & Train Accuracy

In [21]:
Acc_train = nb.score(X_train, y_train)
acc_test = nb.score(X_test, y_test)
print('Train Accuracy : {:.2f}%'.format(Acc_train*100))
print('Test Accuracy  : {:.2f}%'.format(acc_test*100))

# ✔️ Classification report

In [22]:
print(classification_report(y_test, y_pred))

Vocabulary 

In [23]:
vocabulary_list= []
for i in data['headline']:
  sp= i.split(' ')
  for j in sp:
    if j not in vocabulary_list:
      vocabulary_list.append(j)

In [24]:
vocabulary_list

In [ ]:
#Alpha value for smoothing
a = 0.001
#Calculate probability of each word based on class
pb_ij = df.groupby(['classIdx','wordIdx'])
pb_j = df.groupby(['classIdx'])
Pr =  (pb_ij['count'].sum() + a) / (pb_j['count'].sum() + len(vect.vocabulary_))    
#Unstack series
Pr = Pr.unstack()

#Replace NaN or columns with 0 as word count with a/(count+|V|+1)
for c in range(0,41):
    Pr.loc[c,:] = Pr.loc[c,:].fillna(a/(pb_j['count'].sum()[c] + 16689))

#Convert to dictionary for greater speed
Pr_dict = Pr.to_dict()

Pr

In [26]:
#stopwords
print(", ".join(stopwords.words('english')))
stops = set(stopwords.words('english'))

In [ ]:
def calculate_prior_probabilities(df):
    prior_probabilities = df.groupby(by = 'label').apply(lambda x: len(x)/len(df))
    return np.log(prior_probabilities).values
def calculate_posterior_probabilities(df_row, mean, variance, n_unique_labels, n_cols):
    posterior_probabilities = []
  
  # calculate probabilities wrt each label to find max
    for i in range(n_unique_labels):
        posterior = 0

    # for each feature
    for j in range(n_cols):
        posterior += np.log(calculate_probability_density(mean[i][j], variance[i][j], df_row[j]))
    posterior_probabilities.append(posterior)

    return posterior_probabilities

In [ ]:
def predict(nbc,df):
    x_test = df.drop(columns=['label'])
    x_test = x_test.to_numpy()
    predictions = []
    for i in tqdm.tqdm(range(len(x_test))):
        prior = nbc['prior_probabilities']
        posterior = calculate_posterior_probabilities(x_test[i], nbc['mean'], nbc['variance'], nbc['n_unique_labels'], nbc['n_cols'])  # returns log
        probabilities = prior + posterior
        # one with max prob will be the output 
        mx_idx = np.argmax(probabilities)

        predictions.append(nbc['unique_labels'][mx_idx])  # add log values

    return predictions

In [ ]:
nbc = fit(dfb_train)

In [ ]:
# make predictions
predictions = predict(nbc,dfb_test)

In [ ]:
import matplotlib
fig, ax = plt.subplots(figsize=(20,20))

disp = ConfusionMatrixDisplay.from_predictions(dfb_test['label'].to_numpy(),predictions,ax=ax)
plt.grid(False)
plt.show()

In [ ]:
param_grid = { 
    'alpha': np.arange(0, 1.1, 0.1),
    'fit_prior': [True, False],
}

svc = MultinomialNB()
nb_clf = GridSearchCV(estimator=svc, param_grid=param_grid, cv= 5, # Stratified Kfold by default (cv)
                      scoring='balanced_accuracy', refit=True,
                      return_train_score=False, n_jobs=-1, verbose=1)
nb_clf.fit(X_train_enc, y_train)

In [ ]:
def general_metrics(y_test, y_pred):
    print('Accuracy: ' + str(accuracy_score(y_test, y_pred).round(2)))
    print('Balanced Accuracy: ' + str(balanced_accuracy_score(y_test, y_pred).round(2)))
    print('Report:')
    print(classification_report(y_test, y_pred))

def plot_confusion_matrix(y_test, y_pred, classes):
    fig, ax = plt.subplots(figsize=(10, 8))
    cmp = ConfusionMatrixDisplay(
        confusion_matrix(y_test, y_pred),
        display_labels=classes)
    cmp.plot(ax=ax)
    plt.show()
    
def plot_roc_curve(y_test, y_score, classes):
    n_classes = len(classes)
    y_test = label_binarize(y_test, classes=np.arange(n_classes))

    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
  
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    plt.figure(figsize=(10,5))
    lw = 2
    plt.plot(fpr["micro"], tpr["micro"],
             label="micro-average ROC curve (area = {0:0.2f})".format(roc_auc["micro"]),
             color="tab:pink", linestyle=":", linewidth=4,)

    plt.plot(fpr["macro"], tpr["macro"],
             label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
             color="tab:gray", linestyle=":", linewidth=4,)

    colors = ["tab:blue", "tab:orange", "tab:green", "tab:red", "tab:purple"]
    for i, color in enumerate(colors):
        type_class = classes[i]
        plt.plot(fpr[i], tpr[i], color=color, lw=lw, 
                 label="ROC curve of " + type_class +" (area = {1:0.2f})".format(i, roc_auc[i]),)

    plt.plot([0, 1], [0, 1], "k--", lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC curve")
    plt.legend()

References:
https://dataaspirant.com/naive-bayes-classifier-machine-learning/ 


https://www.analyticsvidhya.com/blog/2017/09/naive-bayes-explained/ 


https://www.kaggle.com/datasets/rmisra/news-category-dataset/code (No Contribution)


https://www.kaggle.com/code/hamzamanssor/news-category-classification 
(No Contribution)

https://www.kaggle.com/code/lucasgarciadeviedma/profile-based-retrieval-with-w2vec-and-nn (No Contribution)


https://www.kaggle.com/code/jaimemorillo/profile-based-retrieval 


https://www.kaggle.com/code/tuhinkumardutta/newscategory-lda-topic-modelling-0-41coherence 


 https://towardsdatascience.com/multinomial-naive-bayes-classifier-for-text-analysis-python-8dd6825ece67 


https://machinelearningmastery.com/k-fold-cross-validation/ 


https://towardsdatascience.com/laplace-smoothing-in-na%C3%AFve-bayes-algorithm-9c237a8bdece 
